In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pizza-price-prediction/pizza_v2.csv
/kaggle/input/pizza-price-prediction/pizza_v1.csv


# Importing Necessary Libraries

In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import sklearn.metrics as metrics
import os

# Reading the Dataset

In [3]:
data = pd.read_csv('/kaggle/input/pizza-price-prediction/pizza_v2.csv')

In [4]:
data.head()

,company,price_rupiah,diameter,topping,variant,size,extra_sauce,extra_cheese,extra_mushrooms
0,A,"Rp235,000",22 inch,chicken,double_signature,jumbo,yes,yes,no
1,A,"Rp198,000",20 inch,papperoni,double_signature,jumbo,yes,yes,no
2,A,"Rp120,000",16 inch,mushrooms,double_signature,reguler,yes,yes,yes
3,A,"Rp155,000",14 inch,smoked_beef,double_signature,reguler,yes,no,yes
4,A,"Rp248,000",18 inch,mozzarella,double_signature,jumbo,yes,no,yes


# Analyzing Preparing the Dataset

In [5]:
data.isnull().sum()

company            0
price_rupiah       0
diameter           0
topping            0
variant            0
size               0
extra_sauce        0
extra_cheese       0
extra_mushrooms    0
dtype: int64

### No Missing Values

In [6]:
data.describe

<bound method NDFrame.describe of     company price_rupiah  diameter      topping           variant     size  \
0         A    Rp235,000   22 inch      chicken  double_signature    jumbo   
1         A    Rp198,000   20 inch    papperoni  double_signature    jumbo   
2         A    Rp120,000   16 inch    mushrooms  double_signature  reguler   
3         A    Rp155,000   14 inch  smoked_beef  double_signature  reguler   
4         A    Rp248,000   18 inch   mozzarella  double_signature    jumbo   
..      ...          ...       ...          ...               ...      ...   
124       E     Rp39,000  8.5 inch         tuna        spicy tuna    small   
125       E     Rp72,000   12 inch         tuna        spicy tuna   medium   
126       E     Rp99,000   14 inch         tuna        spicy tuna    large   
127       E     Rp44,000  8.5 inch         meat   BBQ_meat_fiesta    small   
128       E     Rp78,000   12 inch         meat   BBQ_meat_fiesta   medium   

    extra_sauce extra_cheese 

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company          129 non-null    object
 1   price_rupiah     129 non-null    object
 2   diameter         129 non-null    object
 3   topping          129 non-null    object
 4   variant          129 non-null    object
 5   size             129 non-null    object
 6   extra_sauce      129 non-null    object
 7   extra_cheese     129 non-null    object
 8   extra_mushrooms  129 non-null    object
dtypes: object(9)
memory usage: 9.2+ KB


### All object values

### Convering Object data like Price and Diameter to numerical 

In [8]:
data.head()

,company,price_rupiah,diameter,topping,variant,size,extra_sauce,extra_cheese,extra_mushrooms
0,A,"Rp235,000",22 inch,chicken,double_signature,jumbo,yes,yes,no
1,A,"Rp198,000",20 inch,papperoni,double_signature,jumbo,yes,yes,no
2,A,"Rp120,000",16 inch,mushrooms,double_signature,reguler,yes,yes,yes
3,A,"Rp155,000",14 inch,smoked_beef,double_signature,reguler,yes,no,yes
4,A,"Rp248,000",18 inch,mozzarella,double_signature,jumbo,yes,no,yes


In [9]:
data['price_rupiah'] = data['price_rupiah'].apply(lambda x : x.replace("Rp","").replace(",",""))
# remove rp and comma and store it in new price column
data['diameter'] = data['diameter'].apply(lambda x : x.replace(" inch",""))
#remove inch and store it in new column

In [10]:
data.head()

,company,price_rupiah,diameter,topping,variant,size,extra_sauce,extra_cheese,extra_mushrooms
0,A,235000,22,chicken,double_signature,jumbo,yes,yes,no
1,A,198000,20,papperoni,double_signature,jumbo,yes,yes,no
2,A,120000,16,mushrooms,double_signature,reguler,yes,yes,yes
3,A,155000,14,smoked_beef,double_signature,reguler,yes,no,yes
4,A,248000,18,mozzarella,double_signature,jumbo,yes,no,yes


In [11]:
data["diameter"] = data["diameter"].astype(float)
data["price_rupiah"] = data["price_rupiah"].astype(float)

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   company          129 non-null    object 
 1   price_rupiah     129 non-null    float64
 2   diameter         129 non-null    float64
 3   topping          129 non-null    object 
 4   variant          129 non-null    object 
 5   size             129 non-null    object 
 6   extra_sauce      129 non-null    object 
 7   extra_cheese     129 non-null    object 
 8   extra_mushrooms  129 non-null    object 
dtypes: float64(2), object(7)
memory usage: 9.2+ KB


In [13]:
data.head()

,company,price_rupiah,diameter,topping,variant,size,extra_sauce,extra_cheese,extra_mushrooms
0,A,235000.0,22.0,chicken,double_signature,jumbo,yes,yes,no
1,A,198000.0,20.0,papperoni,double_signature,jumbo,yes,yes,no
2,A,120000.0,16.0,mushrooms,double_signature,reguler,yes,yes,yes
3,A,155000.0,14.0,smoked_beef,double_signature,reguler,yes,no,yes
4,A,248000.0,18.0,mozzarella,double_signature,jumbo,yes,no,yes


### Checking for categories under each categorical feature


In [14]:
data['topping'].unique()

array(['chicken', 'papperoni', 'mushrooms', 'smoked_beef', 'mozzarella',
       'black_papper', 'tuna', 'meat', 'sausage', 'onion', 'vegetables',
       'beef'], dtype=object)

In [15]:
data['variant'].unique()

array(['double_signature', 'american_favorite', 'super_supreme',
       'meat_lovers', 'double_mix', 'classic', 'crunchy', 'new_york',
       'double_decker', 'spicy_tuna', 'BBQ_meat_fiesta', 'BBQ_sausage',
       'extravaganza', 'meat_eater', 'gournet_greek', 'italian_veggie',
       'thai_veggie', 'american_classic', 'neptune_tuna', 'spicy tuna'],
      dtype=object)

In [16]:
data['size'].unique()

array(['jumbo', 'reguler', 'small', 'medium', 'large', 'XL'], dtype=object)

In [17]:
data['extra_sauce'].unique()

array(['yes', 'no'], dtype=object)

In [18]:
data['extra_cheese'].unique()

array(['yes', 'no'], dtype=object)

In [19]:
data['extra_mushrooms'].unique()

array(['no', 'yes'], dtype=object)

### Encoding categorical features

In [20]:
en = LabelEncoder()
catCols = ['company','topping','variant','size','extra_sauce','extra_cheese','extra_mushrooms']
for cols in catCols:
    data[cols] = en.fit_transform(data[cols])

In [21]:
data.head()

,company,price_rupiah,diameter,topping,variant,size,extra_sauce,extra_cheese,extra_mushrooms
0,0,235000.0,22.0,2,8,1,1,1,0
1,0,198000.0,20.0,7,8,1,1,1,0
2,0,120000.0,16.0,5,8,4,1,1,1
3,0,155000.0,14.0,9,8,4,1,0,1
4,0,248000.0,18.0,4,8,1,1,0,1


In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   company          129 non-null    int64  
 1   price_rupiah     129 non-null    float64
 2   diameter         129 non-null    float64
 3   topping          129 non-null    int64  
 4   variant          129 non-null    int64  
 5   size             129 non-null    int64  
 6   extra_sauce      129 non-null    int64  
 7   extra_cheese     129 non-null    int64  
 8   extra_mushrooms  129 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 9.2 KB


### Whole dataset is in numerical form now!!
### 
### 


# Splitting into X and Y

In [23]:
Y = data["price_rupiah"]
X = data.drop("price_rupiah",axis = 1)

In [24]:
X.head()

,company,diameter,topping,variant,size,extra_sauce,extra_cheese,extra_mushrooms
0,0,22.0,2,8,1,1,1,0
1,0,20.0,7,8,1,1,1,0
2,0,16.0,5,8,4,1,1,1
3,0,14.0,9,8,4,1,0,1
4,0,18.0,4,8,1,1,0,1


In [25]:
Y.head()

0    235000.0
1    198000.0
2    120000.0
3    155000.0
4    248000.0
Name: price_rupiah, dtype: float64

In [26]:
Y=pd.DataFrame(Y)

In [27]:
Y.head()

,price_rupiah
0,235000.0
1,198000.0
2,120000.0
3,155000.0
4,248000.0


# Test-Train Split

In [28]:
# Break off validation set from training data
X_train_full, X_valid_full, Y_train, Y_valid = train_test_split(X, Y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [29]:
X_train_full.head()

,company,diameter,topping,variant,size,extra_sauce,extra_cheese,extra_mushrooms
120,4,14.0,4,4,2,1,1,0
16,0,16.0,9,13,1,0,1,1
89,3,8.5,4,4,5,1,0,1
119,4,12.0,5,4,3,1,1,0
13,0,12.0,1,18,4,0,1,0


In [30]:
Y_train.head()

,price_rupiah
120,72000.0
16,110000.0
89,23500.0
119,46000.0
13,90000.0


# XGB Regressor

In [31]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
# Define the model
model = XGBRegressor(n_estimators=1000, learning_rate=0.09) 

# Fit the model
model.fit(X_train_full, Y_train) 

# Get predictions
predictions = model.predict(X_valid_full) 

# Calculate MAE
mae = mean_absolute_error(predictions, Y_valid)
print("Mean Absolute Error:" , mae)

#Calculating R2
r2 =  metrics.r2_score(Y_valid, predictions)
print("R2 score :", r2)

Mean Absolute Error: 5551.814002403846
R2 score : 0.9134005216763394


In [32]:
predictions=pd.DataFrame(predictions)

In [33]:
predictions.head()

,0
0,65284.460938
1,149943.203125
2,69741.179688
3,97155.601562
4,73976.429688


# Improved XGB egressor

In [34]:
# Define the model
model2 = XGBRegressor(n_estimators=1000, learning_rate=0.01,max_depth=10) 

# Fit the model
model2.fit(X_train_full, Y_train) 

# Get predictions
predictions2 = model2.predict(X_valid_full) 

# Calculate MAE
mae2 = mean_absolute_error(predictions2, Y_valid)
print("Mean Absolute Error:" , mae2)

#Calculating R2
r22 =  metrics.r2_score(Y_valid, predictions2)
print("R2 score :", r22)

Mean Absolute Error: 5167.389573317308
R2 score : 0.9386071733668582
